In [ ]:
# colab 사용시 드라이브 연결

# from google.colab import drive
# drive.mount('/content/drive')

In [16]:
import pandas as pd
import random
from tqdm import tqdm
import warnings 
import re
warnings.filterwarnings('ignore')

In [9]:
path = '/content/drive/MyDrive/통계데이터센터/'

train = pd.read_csv(path + 'train_preprocessing.csv',encoding='cp949')

In [10]:
train

,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
0,S,95,952,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차 부분정비 타이어 오일 교환
1,G,47,472,상점내에서,일반인을 대상으로,채소 과일판매,상점 내에서 일반인을 대상으로 채소 과일 판매
2,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여 사업체에도 매 공업용 고무를 가지고 합성고무도 매
3,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반 소비자에게 열쇠 잠금장치
4,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학 전 아동보육
...,...,...,...,...,...,...,...
637546,S,96,969,협회 및 단체 수리 및 기타 개인 서비스업,그 외 기타 분류 안된 개인 서비스업,점술 및 유사 서비스업,협회 및 단체 수리 및 기타 개인 서비스업 그 외 기타 분류 안된 개인 서비스업 점...
637547,S,96,969,협회 및 단체 수리 및 기타 개인 서비스업,그 외 기타 분류 안된 개인 서비스업,개인 간병 및 유사 서비스업,협회 및 단체 수리 및 기타 개인 서비스업 그 외 기타 분류 안된 개인 서비스업 개...
637548,S,96,969,협회 및 단체 수리 및 기타 개인 서비스업,그 외 기타 분류 안된 개인 서비스업,결혼 상담 및 준비 서비스업,협회 및 단체 수리 및 기타 개인 서비스업 그 외 기타 분류 안된 개인 서비스업 결...
637549,S,96,969,협회 및 단체 수리 및 기타 개인 서비스업,그 외 기타 분류 안된 개인 서비스업,애완 동물 장묘 및 보호 서비스업,협회 및 단체 수리 및 기타 개인 서비스업 그 외 기타 분류 안된 개인 서비스업 애...


In [11]:
def get_only_hangul(line):
    parseText= re.compile('/ ^[^A-Za-z가-힣]*$/').sub('',line)
    return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    if len(new_words) != 0:
        sentence = ' '.join(new_words)
        new_words = sentence.split(" ")

    else:
        new_words = ""

    return new_words


def get_synonyms(word):
    synomyms = []

    try:
        for syn in wordnet[word]:
            for s in syn:
                synomyms.append(s)
    except:
        pass

    return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)

    return new_words

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words)
    
    return new_words


def add_word(new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1:
        if len(new_words) >= 1:
            random_word = new_words[random.randint(0, len(new_words)-1)]
            synonyms = get_synonyms(random_word)
            counter += 1
        else:
            random_word = ""

        if counter >= 10:
            return
        
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)



def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
    sentence = get_only_hangul(sentence)
    words = sentence.split(' ')
    words = [word for word in words if word is not ""]
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = int(num_aug/4) + 1

    n_sr = max(1, int(alpha_sr*num_words))
    n_ri = max(1, int(alpha_ri*num_words))
    n_rs = max(1, int(alpha_rs*num_words))

# # sr
# for _ in range(num_new_per_technique):
# a_words = synonym_replacement(words, n_sr)
# augmented_sentences.append(' '.join(a_words))

# # ri
# for _ in range(num_new_per_technique):
# a_words = random_insertion(words, n_ri)
# augmented_sentences.append(' '.join(a_words))

    # rs
    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentences.append(" ".join(a_words))

    # rd
    for _ in range(num_new_per_technique):
        a_words = random_deletion(words, p_rd)
        augmented_sentences.append(" ".join(a_words))

    augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
    random.shuffle(augmented_sentences)

    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    augmented_sentences.append(sentence)

    return augmented_sentences

In [14]:
digit_count = train['digit_3'].value_counts().to_dict()
train['digit_count'] = 0
for num in tqdm(range(len(train))):
    for i,j in digit_count.items():
        if train.iloc[num,2] == i:
            train.iloc[num,7] = j
        else:
            pass

100%|███████████████████████████████████████████████████████████████████████| 637551/637551 [1:11:40<00:00, 148.26it/s]


In [17]:
aug = pd.DataFrame(columns=['text','digit_3'])

for i in tqdm(range(len(train))):
    if train.iloc[i,7] <= 1000:
        num = round(1000/train.iloc[i,7])
        l1 = EDA(train.iloc[i,6], num_aug=num)
        adddf = pd.DataFrame()
        adddf['text'] = l1
        adddf['digit_3'] = train.iloc[i,2]
        aug = pd.concat([aug,adddf])
    else:
        pass

100%|████████████████████████████████████████████████████████████████████████| 637551/637551 [02:03<00:00, 5150.36it/s]


In [33]:
aug = pd.concat([train[['text','digit_3']],aug])
aug.reset_index(inplace=True)
aug = aug[['digit_3', 'text']]
aug = aug.drop_duplicates(subset=['text'])


aug.to_csv(path + 'train_preprocessing_aug.csv', index=False, encoding='cp949')

In [35]:
aug

,digit_3,text
0,952,카센터에서 자동차 부분정비 타이어 오일 교환
1,472,상점 내에서 일반인을 대상으로 채소 과일 판매
2,467,절단하여 사업체에도 매 공업용 고무를 가지고 합성고무도 매
3,475,영업점에서 일반 소비자에게 열쇠 잠금장치
4,872,어린이집 보호자의 위탁을 받아 취학 전 아동보육
...,...,...
773842,857,교육 서비스업 교육 지원 교육지원 기타 서비스업 서비스업
773844,861,및 보건업 사회복지 서비스업 병원 종합병원
773848,861,보건업 및 사회복지 서비스업 치과병원
773854,863,보건업 및 사회복지 보건 공중 서비스업 의료업 공중 보건 의료업
